In [77]:
from datetime import datetime,date
from functools import wraps
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, StaleElementReferenceException, InvalidSessionIdException
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse, unquote

import csv
import hashlib
import logging
import os
import requests
import re
import time
import pdb

from pathlib import Path
from typing import Tuple, Union

In [78]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

In [79]:
comune = 'arezzo'
download_path = './pt'
url_comune = 'https://www.comune.arezzo.it/area-tematica/amministrazione-trasparente'

In [80]:
def retry(max_attempts=3, delay=10, exceptions=(Exception,)):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except exceptions as e:
                    attempts += 1
                    logging.info(f"Attempt {attempts} failed:", e)
                    time.sleep(delay)
            raise RuntimeError(f"Function {func.__name__} failed after {max_attempts} attempts")
        return wrapper
    return decorator

In [81]:
extensions = ('.pdf', '.odt', '.xls', '.csv', '.ods', '.xlsx', '.doc', '.zip', '.docx', '.sxc')

# Driver manager

In [82]:
class WebDriverManager:
    def __init__(self):
        self.driver = None

    def start_driver(self, url):
        if not self.driver or (self.is_driver_open() == False):
            opts = Options()
            # start driver
            s = Service(r".\chromedriver.exe")
            prefs = {"download.default_directory": f'{download_path}',
                    "directory_upgrade": True,
                    "profile.default_content_settings.popups": 0}
            opts.add_experimental_option("prefs", prefs)
            self.driver = Chrome(service=s, options=opts)

            # opts = Options()
            # service = Service()

            # prefs = {"download.default_directory": f'{download_path}',
            #         "directory_upgrade": True,
            #         "profile.default_content_settings.popups": 0,
            #         "plugins.always_open_pdf_externally": True  # It will not show PDF directly in chrome
            #         }

            # opts.add_experimental_option('prefs', prefs)
            # opts.add_argument('--no-sandbox')
            # opts.add_argument('--headless')
            # opts.add_argument('--disable-gpu')
            # opts.add_argument('--disable-dev-shm-usage')

            # self.driver = Chrome(service=service,
            #                 options=opts)

            self.driver.get(url)
            self.check_cookies()
            time.sleep(10) # Waiting for main page to load
        return self.driver  # Return the driver instance
        
    def check_cookies(self):
        try: 
            time.sleep(2)
            cookie_button = self.driver.find_element(By.CLASS_NAME, "agree-button.eu-cookie-compliance-secondary-button")

            cookie_button.click()
        except NoSuchElementException:
            pass

    def is_driver_open(self):
        if self.driver:
            try:
                # Access a property or method of the driver
                self.driver.current_url
                return True
            except NoSuchWindowException:
                return False
            except Exception as e:
                logging.info(f'Driver seems to be closed: {e}')
                return False
        return False
    
    def get_driver(self, link):
        if not self.is_driver_open():
            logging.warning(f'Driver is not responding, reopening')
            self.driver = self.start_driver(link)
        else: 
            logging.info('Driver is running, getting items from current section page')
            self.driver.get(link)
            self.check_cookies()

        return self.driver

    def close_driver(self):
        if self.driver:
            self.driver.quit()
            self.driver = None

Initialize webdriver, and clicking on cookies button

In [83]:
driver_manager = WebDriverManager()
driver = driver_manager.get_driver(link=url_comune)

In [84]:
def get_sections_and_subsections():
    sections = {}

    try:
        driver.get(url_comune)
        # Click on expand sections button
        expand_button = driver.find_element(By.ID, 'menu-lista-button')
        expand_button.click()

        body_e = driver.find_element(By.CLASS_NAME, 'nav.navbar-nav.list-group')
        li_elements = body_e.find_elements(By.TAG_NAME, 'li')

        s_names = [li_e.text for li_e in li_elements]
        link_sections = [li_e.find_element(By.TAG_NAME, 'a').get_attribute('href') for li_e in li_elements]

        for s, s_link in zip(s_names, link_sections):
            try:
                # print(f'SECTION: {s}')
                driver.get(s_link)
                time.sleep(1)

                body_e = driver.find_element(By.CLASS_NAME, 'field-items')
                a_elements = body_e.find_elements(By.TAG_NAME, 'a')

                if a_elements:
                    subsections = {}
                    for a_e in a_elements:
                        ss_name = a_e.text.strip()
                        # print(f'\tSUBSECTION: {ss_name}')
                        ss_link = a_e.get_attribute('href')

                        subsections[ss_name] = ss_link

                    sections[s] = {'href': s_link, **subsections}
                else:
                    sections[s] = {'href': s_link}
            except NoSuchElementException:
                pass
            except Exception as e:
                logging.warning(f'Could not get subsections for {s}: {e}')
                pass

    except Exception as e:
        logging.warning(f'Could not get sections')

    return sections

In [85]:
# sections = get_sections_and_subsections()

In [86]:
# sections

# Scrapping a given subsection

In [87]:
def rename_file(file_name:str, link_ogg: str)->str:
    # Generate a unique identifier for the file
    id = hashlib.sha256(link_ogg.encode()).hexdigest()[:8]

    return f'{id}_{file_name}'

In [88]:
def download_file(file_name:str, link_file:str)->Path:
    # Download the file
    response = requests.get(link_file)

    # Construct the full path of the file
    file_path = Path(download_path, file_name)

    # Save the file
    with open(file_path, 'wb') as f:
        f.write(response.content)

    print(file_name)
    
    return file_path

In [89]:
def download_accordion():
    # Expanding attachment menu
    menu_e = driver.find_element(By.ID, 'ui-accordion-1-header-0').find_element(By.TAG_NAME, 'a')
    driver.execute_script("arguments[0].click();", menu_e)

    allegati_e = driver.find_element(By.CLASS_NAME, 'group-documenti.field-group-html-element.documenti')
    file_elements = allegati_e.find_elements(By.CLASS_NAME, 'file')

    logging.info(f'Found {len(file_elements)} files to download')
    for f_e in file_elements:
        a_e = f_e.find_element(By.TAG_NAME, 'a')
        link_file = a_e.get_attribute('href')
        if link_file.endswith(extensions):
            try:
                # Get the last part of the resulting path
                link_parts = link_file.split('/')
                file_name = rename_file(file_name=link_parts[-1],
                                        link_ogg=link_file)
                
                download_file(file_name=file_name,
                            link_file=link_file)
            except Exception as e:
                logging.warning(f'Could not download file {link_file}: {e}')
        else:
            pass

In [90]:
def download_required_fields():
    # body_e = driver.find_element(By.CLASS_NAME, 'required-fields.group-documenti.field-group-html-element.documenti')
    file_elements = driver.find_elements(By.CLASS_NAME, 'file')

    logging.info(f'Found {len(file_elements)} files to download')
    for f_e in file_elements:
        a_e = f_e.find_element(By.TAG_NAME, 'a')
        link_file = a_e.get_attribute('href')
        if link_file.endswith(extensions):
            try:
                # Get the last part of the resulting path
                link_parts = link_file.split('/')
                file_name = rename_file(file_name=link_parts[-1],
                                        link_ogg=link_file)
                
                download_file(file_name=file_name, link_file=link_file)
            except Exception as e:
                logging.warning(f'Could not download file {link_file}: {e}')
        else:
            pass


In [91]:
# def download_label_hidden():
#     body_e = driver.find_element(By.CLASS_NAME, 'field.field-name-body.field-type-text-with-summary.field-label-hidden')
#     file_elements = body_e.find_elements(By.TAG_NAME, 'li')

#     for f_e in file_elements:
#         a_e = f_e.find_element(By.TAG_NAME, 'a')
#         link_file = a_e.get_attribute('href')
#         try:
#             # Get the last part of the resulting path
#             link_parts = link_file.split('/')
#             file_name = rename_file(file_name=link_parts[-1],
#                                     link_ogg=link_file)
            
#             if 'schede' not in link_file:
#                 download_file(file_name=file_name, link_file=link_file)
#             else: 
#                 download_accordion()
#         except Exception as e:
#             print(e)        

In [92]:
# def download_files_from_microsection(link_microsection: str)-> None:
#     driver.get(link_microsection)

#     try:
#         download_accordion()
#     except:
#         logging.info(f'Could not download files from accordion')
#     try: 
#         download_label_hidden()
#     except:
#         logging.info(f'Could not find files by label hidden')
#     try: 
#         download_required_fields()
#     except:
#         logging.info(f'Could not find files by required fields')

In [93]:
def get_microsections_field_items(link_ss):
    subsections = {}
    try:
        driver.get(link_ss)
        body_e = driver.find_element(By.CLASS_NAME, 'field-items')
        a_elements = body_e.find_elements(By.TAG_NAME, 'a')

        for a_e in a_elements:
            ms_link = a_e.get_attribute('href')

            subsections[ms_link] = {'href': ms_link}
    except:
        pass

    return subsections

In [94]:
get_microsections_field_items('https://www.comune.arezzo.it/area-tematica/polizia-locale-sicurezza')

{}

In [95]:
def get_microsections_bandi(link_ss: str):
    try: 
        driver.get(link_ss)
        driver.find_element(By.CLASS_NAME, "view-content")
    except NoSuchElementException:
        return {}
    
    page_number = 0
    subsections = {}
    previous_links = set()
    
    while True:
        driver.get(f'{link_ss}?page={page_number}')
    
        try: 
            body_e = driver.find_element(By.CLASS_NAME, "view-content")
        except NoSuchElementException:
            return subsections
        
        content_e = body_e.find_elements(By.CLASS_NAME, 'views-field.views-field-title')

        if content_e:
            current_links = set()
            for c_e in content_e:
                a_e = c_e.find_element(By.TAG_NAME, 'a')
                ms_link = a_e.get_attribute('href')
                
                current_links.add(ms_link)
                subsections[ms_link] = {'href': ms_link}
            
            # Check if current page links are the same as previous page links
            if current_links == previous_links:
                break
            previous_links = current_links
        else:
            return subsections
        
        page_number += 1
    
    return subsections


In [96]:
get_microsections_bandi('https://www.comune.arezzo.it/certificazioni-anagrafiche')

{}

In [97]:
def get_microsections_cards(link_ss: str):
    try: 
        driver.get(link_ss)
        driver.find_element(By.ID, "views-bootstrap-grid-1")
    except NoSuchElementException:
        return {}
    
    page_number = 0
    subsections = {}
    previous_links = set()
    
    while True:
        driver.get(f'{link_ss}?page={page_number}')
        print(f'{link_ss}?page={page_number}')
    
        try:
            body_e = driver.find_element(By.ID, "views-bootstrap-grid-1")
        except NoSuchElementException:
            return subsections
        
        cards_e = body_e.find_elements(By.CLASS_NAME, 'views-field.views-field-title')

        if cards_e:
            for c_e in cards_e:
                current_links = set()
                a_e = c_e.find_element(By.TAG_NAME, 'a')
                ms_link = a_e.get_attribute('href')

                current_links.add(ms_link)
                print(current_links)
                subsections[ms_link] = {'href': ms_link}

                # Check if current page links are the same as previous page links
                if current_links == previous_links:
                    break
                previous_links = current_links
        else:
            return subsections
        
        page_number += 1

In [98]:
get_microsections_cards('https://www.comune.arezzo.it/certificazioni-anagrafiche')

https://www.comune.arezzo.it/certificazioni-anagrafiche?page=0


{}

In [99]:
def get_all_types_microsections(link_ss):
    driver.get(link_ss)
    # sections = {}

    sections = get_microsections_field_items(link_ss)
    if not sections:
        sections = get_microsections_cards(link_ss)
        if not sections:
            sections = get_microsections_bandi(link_ss)

    return sections

In [106]:
links_visited = ['https://www.comune.arezzo.it']
def get_recursive_microsections(link, depth=0, max_depth=10):
    if depth > max_depth:
        return

    microsections = get_all_types_microsections(link)
    print(microsections)
    if microsections:
        for values in microsections.values():
            link = values['href']
            if link and ('https://www.comune.arezzo.it' in link) and (link not in links_visited):
                if link.endswith(extensions):
                    # Get the last part of the resulting path
                    link_parts = link.split('/')
                    file_name = rename_file(file_name=link_parts[-1],
                                            link_ogg=link)
                    download_file(file_name=file_name,
                                  link_file=link)
                    
                    links_visited.append(link)
                else: 
                    print(link)
                    try:
                        download_accordion()
                    except:
                        pass
                    try:
                        download_required_fields()
                    except:
                        pass

                    links_visited.append(link)
                    get_recursive_microsections(link, depth + 1, max_depth)
            else: 
                pass

In [109]:
get_recursive_microsections('http://www.comune.arezzo.it/altri-contenuti')

INFO:root:Found 0 files to download


{'https://www.comune.arezzo.it/raccolta-comunale-delle-informative-gdpr': {'href': 'https://www.comune.arezzo.it/raccolta-comunale-delle-informative-gdpr'}, 'https://www.comune.arezzo.it/controllo-successivo-regolarita-amministrativa': {'href': 'https://www.comune.arezzo.it/controllo-successivo-regolarita-amministrativa'}, 'https://www.comune.arezzo.it/piani-razionalizzazione': {'href': 'https://www.comune.arezzo.it/piani-razionalizzazione'}, 'http://www3.comune.arezzo.it/il-comune/servizio-sociale-politiche-educative-formative-sport/processi-partecipativi': {'href': 'http://www3.comune.arezzo.it/il-comune/servizio-sociale-politiche-educative-formative-sport/processi-partecipativi'}, 'https://www.comune.arezzo.it/pubblicazione-bilanci-consuntivi-sagre-destinazioni-proventi': {'href': 'https://www.comune.arezzo.it/pubblicazione-bilanci-consuntivi-sagre-destinazioni-proventi'}}
https://www.comune.arezzo.it/raccolta-comunale-delle-informative-gdpr


INFO:root:Found 0 files to download


{}
https://www.comune.arezzo.it/controllo-successivo-regolarita-amministrativa
{'https://www.comune.arezzo.it/sites/default/files/modello_relazione_annuale_2021_controlli_succ_18_gen_2022_prot.pdf': {'href': 'https://www.comune.arezzo.it/sites/default/files/modello_relazione_annuale_2021_controlli_succ_18_gen_2022_prot.pdf'}, 'https://www.comune.arezzo.it/sites/default/files/relazione_i_trimestre_2022_controlli_prot.pdf': {'href': 'https://www.comune.arezzo.it/sites/default/files/relazione_i_trimestre_2022_controlli_prot.pdf'}, 'https://www.comune.arezzo.it/sites/default/files/relazione_ii_trimestre_2022_controlli_prot.pdf': {'href': 'https://www.comune.arezzo.it/sites/default/files/relazione_ii_trimestre_2022_controlli_prot.pdf'}, 'https://www.comune.arezzo.it/sites/default/files/relazione_iii_trimestre_2022_controlli_prot.pdf': {'href': 'https://www.comune.arezzo.it/sites/default/files/relazione_iii_trimestre_2022_controlli_prot.pdf'}, 'https://www.comune.arezzo.it/sites/default/file

INFO:root:Found 0 files to download


...

{'https://www.comune.arezzo.it/delibera-gc-n-112-del-25032019': {'href': 'https://www.comune.arezzo.it/delibera-gc-n-112-del-25032019'}, 'http://www.comune.arezzo.it/sites/default/files/allegato_a_3_0.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/allegato_a_3_0.pdf'}, 'http://www.comune.arezzo.it/sites/default/files/allegato_b_2_0.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/allegato_b_2_0.pdf'}, 'http://www.comune.arezzo.it/sites/default/files/allegato_c_3_0.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/allegato_c_3_0.pdf'}, 'http://www.comune.arezzo.it/sites/default/files/allegato_d_2_0.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/allegato_d_2_0.pdf'}}
https://www.comune.arezzo.it/delibera-gc-n-112-del-25032019
{'https://www.comune.arezzo.it/sites/default/files/gc_2019_112.odt': {'href': 'https://www.comune.arezzo.it/sites/default/files/gc_2019_112.odt'}}


INFO:root:Found 1 files to download


c14e201f_gc_2019_112.odt
https://www.comune.arezzo.it/pubblicazione-bilanci-consuntivi-sagre-destinazioni-proventi


INFO:root:Found 1 files to download


c14e201f_gc_2019_112.odt
c14e201f_gc_2019_112.odt
{'http://www.comune.arezzo.it/sites/default/files/sagre_e_feste_paesane_-_-2016-_rendiconto_consuntivo.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/sagre_e_feste_paesane_-_-2016-_rendiconto_consuntivo.pdf'}, 'http://www.comune.arezzo.it/sites/default/files/sagre_e_feste_paesane_-_-2017-_bilancio_preventivo.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/sagre_e_feste_paesane_-_-2017-_bilancio_preventivo.pdf'}, 'http://www.comune.arezzo.it/sites/default/files/sagre.prev_.2018.cons_.2017.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/sagre.prev_.2018.cons_.2017.pdf'}, 'http://www.comune.arezzo.it/sites/default/files/sagre_bilanci_cons._2018_e_prev._2019_-_pubbl.pdf': {'href': 'http://www.comune.arezzo.it/sites/default/files/sagre_bilanci_cons._2018_e_prev._2019_-_pubbl.pdf'}}


# Method to get all bandi di concorso attivi

In [102]:
sections = get_microsections_bandi('https://www.comune.arezzo.it/concorsi')

In [103]:
sections

{'https://www.comune.arezzo.it/bandoastaconcorso/interpello-prova-unica-orale-finalizzato-allassunzione-tempo-pieno-indeterminato-1': {'href': 'https://www.comune.arezzo.it/bandoastaconcorso/interpello-prova-unica-orale-finalizzato-allassunzione-tempo-pieno-indeterminato-1'},
 'https://www.comune.arezzo.it/bandoastaconcorso/concorso-pubblico-esami-copertura-contratto-tempo-pieno-ed-indeterminato-n-9-posti': {'href': 'https://www.comune.arezzo.it/bandoastaconcorso/concorso-pubblico-esami-copertura-contratto-tempo-pieno-ed-indeterminato-n-9-posti'},
 'https://www.comune.arezzo.it/bandoastaconcorso/interpello-prova-unica-orale-finalizzato-alla-copertura-tempo-pieno-0': {'href': 'https://www.comune.arezzo.it/bandoastaconcorso/interpello-prova-unica-orale-finalizzato-alla-copertura-tempo-pieno-0'},
 'https://www.comune.arezzo.it/bandoastaconcorso/avviso-manifestazione-interesse-candidatura-presidente-del-collegio-dei-revisori-0': {'href': 'https://www.comune.arezzo.it/bandoastaconcorso/avvi

In [104]:
def download_bandi():
    # body_e = driver.find_element(By.CLASS_NAME, 'view-content')
    file_elements = driver.find_elements(By.CLASS_NAME, 'file')
    for f_e in file_elements:
        a_e = f_e.find_element(By.TAG_NAME, 'a')
        link_file = a_e.get_attribute('href')
        if link_file.endswith(extensions):
            try:
                # Get the last part of the resulting path
                link_parts = link_file.split('/')
                file_name = rename_file(file_name=link_parts[-1],
                                        link_ogg=link_file)
                
                download_file(file_name=file_name, link_file=link_file)
            except Exception as e:
                print(e)
        else:
            pass

In [105]:
# for ms, values in sections.items():
#     link = values['href']

#     driver.get(link)
#     download_bandi()